In [ ]:
class RB(nn.Module):
    """ initial manual block """
    def __init__(self, in_channels, out_channels, stride=1, padding=1, kernel=3):
        super().__init__()
        # one by one
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding)
        self.conv1_bn = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding)
        self.conv2_bn = nn.BatchNorm2d(out_channels)
        self.residual = None
        if in_channels != out_channels:
            self.residual = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv1_bn(out)
        out = F.relu(out) # can move to module and use nn.relu
        out = self.conv2(out)
        out = self.conv2_bn(out)
        if self.residual is not None:
            out += self.residual(x)
        else:
            out += x
        out = F.relu(out)
        return out

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, padding=1, kernel=3):
        super().__init__()
        # sequential
        self.blocks = nn.Sequential(conv_block(in_channels, out_channels, kernel, stride, padding),
                                    nn.ReLU(),
                                    conv_block(out_channels, out_channels, kernel, stride, padding)
                                    )
        self.residual = None
        if in_channels != out_channels:
            self.residual = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)

    def forward(self, x):
        out = self.blocks(x)
        if self.residual is not None:
            out += self.residual(x)
        else:
            out += x
        out = F.relu(out)
        return out


def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(nn.Conv2d(in_f, out_f, *args, **kwargs),
                         nn.BatchNorm2d(out_f)
                         )

class RBPool(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, padding=1, kernel=3, pool='max'):
        super().__init__()
        self.pool = nn.Identity()
        if pool == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pool == 'avg':
            self.pool = nn.AdaptiveAvgPool2d((1, 1))

        self.rb_pool = nn.Sequential(ResidualBlock(in_channels, out_channels, stride=1, padding=1, kernel=3),
                                     self.pool
                                     )

    def forward(self, x):
        return self.rb_pool(x)

class Ex1Net(nn.Module):
    def __init__(self, in_channels, out_channels, pools, num_classes):
        super().__init__()
        num_layers = len(in_channels)
        # using ModuleList
        # self.layers = nn.ModuleList([RBPool(in_channels[i], out_channels[i], pools[i]) for i in range(num_layers)])

        # using Sequential and zip
        layers = [RBPool(in_c, out_c, pool=p)
                  for in_c, out_c, p in
                  zip(in_channels, out_channels, pools)]
        self.layers = nn.Sequential(*layers)

        self.linear = nn.Linear(out_channels[-1], num_classes)
        self.linear_in_dim = out_channels[-1]

    def forward(self, x):
        # # for moduleList
        # for layer in self.layers:
        #     x = layer(x)

        # # for sequential
        x = self.layers(x)

        # Hint: need to reshape before applying FC
        x = x.reshape(-1, self.linear_in_dim)  # flat for FC, the size -1 is inferred from other dimensions
        x = self.linear(x)
        return x


